In [20]:
import pandas as pd
electric_utility_data = pd.read_csv("electric.csv")
electric_utility_data_1st_four_yrs = electric_utility_data[electric_utility_data["report_year"] <= 1998]

electric_utility_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'electric.csv'

In [ ]:
electric_utility_data_1st_four_yrs.head()

,record_id,utility_id_ferc1,report_year,plant_name_ferc1,fuel_type_code_pudl,fuel_unit,fuel_qty_burned,fuel_mmbtu_per_unit,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_cost_per_mmbtu
0,f1_fuel_1994_12_1_0_7,1,1994,rockport,coal,ton,5377489.0,16.590,18.59,18.53,1.121
1,f1_fuel_1994_12_1_0_10,1,1994,rockport total plant,coal,ton,10486945.0,16.592,18.58,18.53,1.120
2,f1_fuel_1994_12_2_0_1,2,1994,gorgas,coal,ton,2978683.0,24.130,39.72,38.12,1.650
3,f1_fuel_1994_12_2_0_7,2,1994,barry,coal,ton,3739484.0,23.950,47.21,45.99,1.970
4,f1_fuel_1994_12_2_0_10,2,1994,chickasaw,gas,mcf,40533.0,1.000,2.77,2.77,2.570


The dataset for the remainder of this quiz is the Appliances Energy Prediction data. 
... The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. 
... Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods.
... The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) 
... was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. 
... Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). 
... The attribute information can be seen below.

# Attribute Information:

' Date, time year-month-day hour:minute:second

Appliances, energy use in Wh

lights, energy use of light fixtures in the house in Wh

T1, Temperature in kitchen area, in Celsius

"RH_1": Humidityin kitchen area, in %

T2, Temperature in living room area, in Celsius

RH_2, Humidity in living room area, in %

T3, Temperature in laundry room area

RH_3, Humidity in laundry room area, in %

T4, Temperature in office room, in Celsius

RH_4, Humidity in office room, in %

T5, Temperature in bathroom, in Celsius

RH_5, Humidity in bathroom, in %

T6, Temperature outside the building (north side), in Celsius

RH_6, Humidity outside the building (north side), in %

T7, Temperature in ironing room , in Celsius

RH_7, Humidity in ironing room, in %

T8, Temperature in teenager room 2, in Celsius

RH_8, Humidity in teenager room 2, in %

T9, Temperature in parents room, in Celsius

RH_9, Humidity in parents room, in %

To, Temperature outside (from Chievres weather station), in Celsius

Pressure (from Chievres weather station), in mm Hg

RH_out, Humidity outside (from Chievres weather station), in %

Wind speed (from Chievres weather station), in m/s

Visibility (from Chievres weather station), in km

Tdewpoint (from Chievres weather station), Â °C

rv1, Random variable 1, nondimensional

rv2, Random variable 2, nondimensional

To answer some questions, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.


In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

column_names = { "T1":"Temperature_in_kitchen_area", 'RH_1' : 'Humidity_in_kitchen_area',
'T2' : 'Temperature_in_living_room_area', 'RH_2' : 'Humidity_in_living_room_area', 
'T3' : 'Temperature_in_laundry_room_area' ,'RH_3' : 'Humidity_in_laundry_room_area' , 
'T4' : 'Temperature_in_office_room' , 'RH_4' : 'Humidity_in_office_room_area' ,
'T5' : 'Temperature_in_bathroom' , 'RH_5' : 'Humidity_in_bathroom' ,
'T6' : 'NorthSide_Temperature_outside_the_building', 'RH_6':'NorthSide_Temperature_outside_the_building',
'T7':'Temperature_in_iron_room', 'RH_7':'Humidity_in_iron_room', 
'T8':'Temperature_in_teenager_room', 'RH_8':'Humidity_in_teenager_room', 
'T9':'Temperature_in_parents_room', 'RH_7':'Humidity_in_parents_room',
'To' : 'Chievers_station_Temperature_outside_the_building', 'RH_6':'Chivers_airport_Temperature_outside_the_building'}

data = pd.read_csv("energydata_complete.csv")

print(data.head())


                  date  Appliances  lights     T1       RH_1    T2       RH_2  \
0  2016-01-11 17:00:00          60      30  19.89  47.596667  19.2  44.790000   
1  2016-01-11 17:10:00          60      30  19.89  46.693333  19.2  44.722500   
2  2016-01-11 17:20:00          50      30  19.89  46.300000  19.2  44.626667   
3  2016-01-11 17:30:00          50      40  19.89  46.066667  19.2  44.590000   
4  2016-01-11 17:40:00          60      40  19.89  46.333333  19.2  44.530000   

      T3       RH_3         T4  ...         T9   RH_9     T_out  Press_mm_hg  \
0  19.79  44.730000  19.000000  ...  17.033333  45.53  6.600000        733.5   
1  19.79  44.790000  19.000000  ...  17.066667  45.56  6.483333        733.6   
2  19.79  44.933333  18.926667  ...  17.000000  45.50  6.366667        733.7   
3  19.79  45.000000  18.890000  ...  17.000000  45.40  6.250000        733.8   
4  19.79  45.000000  18.890000  ...  17.000000  45.40  6.133333        733.9   

   RH_out  Windspeed  Visibility

In [24]:
data = data.rename(columns=column_names)

print(data.head())


                  date  Appliances  lights  Temperature_in_kitchen_area  \
0  2016-01-11 17:00:00          60      30                        19.89   
1  2016-01-11 17:10:00          60      30                        19.89   
2  2016-01-11 17:20:00          50      30                        19.89   
3  2016-01-11 17:30:00          50      40                        19.89   
4  2016-01-11 17:40:00          60      40                        19.89   

   Humidity_in_kitchen_area  Temperature_in_living_room_area  \
0                 47.596667                             19.2   
1                 46.693333                             19.2   
2                 46.300000                             19.2   
3                 46.066667                             19.2   
4                 46.333333                             19.2   

   Humidity_in_living_room_area  Temperature_in_laundry_room_area  \
0                     44.790000                             19.79   
1                     44.7

In [25]:
data = data.drop(columns=["date","lights"])
print(data)


       Appliances  Temperature_in_kitchen_area  Humidity_in_kitchen_area  \
0              60                    19.890000                 47.596667   
1              60                    19.890000                 46.693333   
2              50                    19.890000                 46.300000   
3              50                    19.890000                 46.066667   
4              60                    19.890000                 46.333333   
...           ...                          ...                       ...   
19730         100                    25.566667                 46.560000   
19731          90                    25.500000                 46.500000   
19732         270                    25.500000                 46.596667   
19733         420                    25.500000                 46.990000   
19734         430                    25.500000                 46.600000   

       Temperature_in_living_room_area  Humidity_in_living_room_area  \
0              

In [26]:
print(data.describe())

         Appliances  Temperature_in_kitchen_area  Humidity_in_kitchen_area  \
count  19735.000000                 19735.000000              19735.000000   
mean      97.694958                    21.686571                 40.259739   
std      102.524891                     1.606066                  3.979299   
min       10.000000                    16.790000                 27.023333   
25%       50.000000                    20.760000                 37.333333   
50%       60.000000                    21.600000                 39.656667   
75%      100.000000                    22.600000                 43.066667   
max     1080.000000                    26.260000                 63.360000   

       Temperature_in_living_room_area  Humidity_in_living_room_area  \
count                     19735.000000                  19735.000000   
mean                         20.341219                     40.420420   
std                           2.192974                      4.069813   
min      